In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import torch.nn.functional as F
import pickle

# Loading models and data

## CNN Reduced Full

In [9]:
# Assuming you have loaded or defined your model
CNN_reduced_full = tf.keras.models.load_model('./models/CNN_b_reduced.h5')

# Load the test data
X_test_CNN_reduced_full = np.load('./data/X_test_CNN_reduced_full.npy')
y_test_CNN_reduced_full = np.load('./data/y_test_encoded_CNN_reduced_full.npy')

## CNN Reduced

In [6]:
CNN_reduced = tf.keras.models.load_model('./models/CNN_d_reduced.h5')

X_test_CNN_reduced = np.load('./data/X_test_CNN_reduced.npy')
y_test_CNN_reduced = np.load('./data/y_test_encoded_CNN_reduced.npy')

## CNN Combined

In [2]:
CNN_combined = tf.keras.models.load_model('./models/CNN.h5')

X_test_CNN_combined = np.load('./data/X_test_CNN_combined.npy')
y_test_CNN_combined = np.load('./data/y_test_encoded_CNN_combined.npy')



## RNN Reduced Full

In [11]:
RNN_reduced_full = tf.keras.models.load_model('./models/RNN_reduced_full.h5')

X_test_RNN_reduced_full = np.load('./data/X_test_RNN_reduced_full.npy')
y_test_RNN_reduced_full = np.load('./data/y_test_RNN_reduced_full.npy')

## RNN Reduced

In [12]:
RNN_reduced = tf.keras.models.load_model('./models/RNN_reduced.h5')

X_test_RNN_reduced = np.load('./data/X_test_RNN_reduced.npy')
y_test_RNN_reduced = np.load('./data/y_test_RNN_reduced.npy')

## RNN Combined

In [13]:
RNN_combined = tf.keras.models.load_model('./models/RNN_combined.h5')

X_test_RNN_combined = np.load('./data/X_test_RNN_combined.npy')
y_test_RNN_combined = np.load('./data/y_test_RNN_combined.npy')

## distilBERT

In [7]:
save_directory = '../saved_BERT'

# Load DistilBERT model and tokenizer
tokenizer_BERT = transformers.DistilBertTokenizer.from_pretrained(save_directory)
model_BERT = transformers.DistilBertForSequenceClassification.from_pretrained(save_directory)

In [8]:

test_df = pd.read_csv('./data/BERT.csv')

# Create test_texts and test_labels as lists

test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

## RFC Combined

In [3]:
# Load model from rfc_model_combined.pkl
rfc_model_combined = pickle.load(open('./models/rfc_model_combined.pkl', 'rb'))

X_test_RFC = np.load('./data/X_test_RFC.npy')
y_test_RFC = np.load('./data/y_test_encoded_RFC.npy')

# Inferencing 

In [3]:
# Function to perform inference without tracking gradients
def inference_step(inputs, loaded_model):
    # Set training to False for layers like Dropout or BatchNormalization
    # to ensure they are in inference mode
    outputs = loaded_model(inputs, training=False)
    return outputs


In [4]:
def inference(idx, loaded_model, X_test, y_test):
    # Perform inference on a single image
    inference_input = X_test[idx:idx+1]
    inference_output = inference_step(inference_input, loaded_model)
    # Get the predicted class with the highest probability
    inference_prediction = np.argmax(inference_output, axis=1)
    # Get the ground truth label
    inference_truth = y_test[idx]
    return inference_prediction, inference_truth

## CNN

In [11]:
pred, truth = inference(1749, CNN_reduced_full, X_test_CNN_reduced_full, y_test_CNN_reduced_full)

print('Prediction: {}'.format(pred))
print('Ground truth: {}'.format(truth))

Prediction: [5]
Ground truth: 5


In [12]:
pred, truth = inference(1749, CNN_reduced, X_test_CNN_reduced, y_test_CNN_reduced)

print('Prediction: {}'.format(pred))
print('Ground truth: {}'.format(truth))

Prediction: [10]
Ground truth: 10


In [13]:
pred, truth = inference(1749, CNN_combined, X_test_CNN_combined, y_test_CNN_combined)

print('Prediction: {}'.format(pred))
print('Ground truth: {}'.format(truth))

Prediction: [3]
Ground truth: 8


## RNN

In [22]:
pred, truth = inference(32, RNN_reduced_full, X_test_RNN_reduced_full, y_test_RNN_reduced_full)

print('Prediction: {}'.format(pred))
print('Ground truth: {}'.format(truth))

Prediction: [5]
Ground truth: 5


In [21]:
pred, truth = inference(32, RNN_reduced, X_test_RNN_reduced, y_test_RNN_reduced)

print('Prediction: {}'.format(pred))
print('Ground truth: {}'.format(truth))

Prediction: [10]
Ground truth: 10


In [23]:
pred, truth = inference(32, RNN_combined, X_test_RNN_combined, y_test_RNN_combined)

print('Prediction: {}'.format(pred))
print('Ground truth: {}'.format(truth))

Prediction: [8]
Ground truth: 8


## distilBERT

In [25]:
idx = 32
test_text = test_texts[idx]

predict_input = tokenizer_BERT.encode_plus(
    test_text,
    truncation=True,
    padding=True,
    return_tensors='pt'
)

output = model_BERT(**predict_input)[0]
prediction_value = torch.argmax(output, axis=1).item()
print('Prediction: {}'.format(prediction_value))
print('Ground truth: {}'.format(test_labels[idx]))

Prediction: 5
Ground truth: 5


## RFC

In [9]:
# Making predictions on the test data
predictions = rfc_model_combined.predict(X_test_RFC[32:33])

print('Prediction: {}'.format(predictions[0]))
print('Ground truth: {}'.format(y_test_RFC[32]))

Prediction: 6
Ground truth: 5


In [ ]:
# Delete all models

del CNN_reduced_full
del CNN_reduced
del CNN_combined
del RNN_reduced_full
del RNN_reduced
del RNN_combined
del rfc_model_combined
del tokenizer_BERT
del model_BERT